In [0]:
dbutils.fs.mount(
  source = "wasbs://retail@retailproject22.blob.core.windows.net",
  mount_point = "/mnt/retail_project07",
  extra_configs = {"fs.azure.account.key.retailproject22.blob.core.windows.net":"BkaHMrldNa+8eSXqdpua7jDS6uXKQFkL8m8sKCoSvx8qQKnJ0xE9PZuvqPjhCJTX8Yvm5iZY+8q++AStbmcQWw=="})

In [0]:
dbutils.fs.ls('/mnt/retail_project/')

[FileInfo(path='dbfs:/mnt/retail_project/bronze/', name='bronze/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/retail_project/gold/', name='gold/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/retail_project/silver/', name='silver/', size=0, modificationTime=0)]

In [0]:
dbutils.fs.ls('/mnt/retail_project/bronze/')

[FileInfo(path='dbfs:/mnt/retail_project/bronze/customer/', name='customer/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/retail_project/bronze/product/', name='product/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/retail_project/bronze/store/', name='store/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/retail_project/bronze/transaction/', name='transaction/', size=0, modificationTime=0)]

In [0]:
dbutils.fs.ls('/mnt/retail_project/bronze/transcation')

In [0]:
df_transactions = spark.read.parquet('/mnt/retail_project/bronze/transaction/')
df_products = spark.read.parquet('/mnt/retail_project/bronze/product/')
df_stores = spark.read.parquet('/mnt/retail_project/bronze/store/')
df_customers = spark.read.parquet('/mnt/retail_project/bronze/customer/naveenkothi-de/Retail-project/refs/heads/main/')
display(df_transactions)
display(df_products)
display(df_stores)
display(df_customers)




In [0]:
from pyspark.sql.functions import col

In [0]:
df_transactions = df_transactions.select(
    col("transaction_id").cast("int"),
    col("customer_id").cast("int"),
    col("product_id").cast("int"),
    col("store_id").cast("int"),
    col("quantity").cast("int"),
    col("transaction_date").cast("date")
)

df_products = df_products.select(
    col("product_id").cast("int"),
    col("product_name"),
    col("category"),
    col("price").cast("double")
)

df_stores = df_stores.select(
    col("store_id").cast("int"),
    col("store_name"),
    col("location")
)

df_customers = df_customers.select(
    "customer_id", "first_name", "last_name", "email", "city", "registration_date"
).dropDuplicates(["customer_id"])


In [0]:
df_silver = df_transactions \
    .join(df_customers, "customer_id") \
    .join(df_products, "product_id") \
    .join(df_stores, "store_id") \
    .withColumn("total_amount", col("quantity") * col("price"))

In [0]:
display(df_silver)

In [0]:
silver_path = "/mnt/retail_project/silver/"

df_silver.write.mode("overwrite").format("delta").save(silver_path)

In [0]:
spark.sql(f"""
CREATE TABLE retail_silver_cleaned
USING DELTA
LOCATION '/mnt/retail_project/silver/'
""")

In [0]:
%sql select * from retail_silver_cleaned

In [0]:
silver_df = spark.read.format("delta").load("/mnt/retail_project/silver/")

In [0]:
from pyspark.sql.functions import sum, countDistinct, avg

gold_df = silver_df.groupBy(
    "transaction_date",
    "product_id", "product_name", "category",
    "store_id", "store_name", "location"
).agg(
    sum("quantity").alias("total_quantity_sold"),
    sum("total_amount").alias("total_sales_amount"),
    countDistinct("transaction_id").alias("number_of_transactions"),
    avg("total_amount").alias("average_transaction_value")
)

In [0]:
display(gold_df)

In [0]:
gold_path = "/mnt/retail_project/gold/"

gold_df.write.mode("overwrite").format("delta").save(gold_path)

In [0]:
spark.sql("""
CREATE TABLE retail_gold_sales_summary
USING DELTA
LOCATION '/mnt/retail_project/gold/' """)


In [0]:
%sql select * from retail_gold_sales_summary

transaction_date,product_id,product_name,category,store_id,store_name,location,total_quantity_sold,total_sales_amount,number_of_transactions,average_transaction_value
2025-04-30,9,Dumbells,Fitness,4,Downtown Store,USA,2,2598.0,1,2598.0
2024-07-17,1,Speaker,Electronics,4,Downtown Store,USA,5,334.95,1,334.95
2024-10-08,1,Speaker,Electronics,3,Gachibowli Outlet,Hyderabad,2,133.98,1,133.98
2024-07-14,8,Webcam,Accessories,4,Downtown Store,USA,4,356.0,1,356.0
2024-07-30,5,Books,Stationery,1,Sarah Mall Store,Hyderabad,2,200.0,1,200.0
2024-12-13,8,Webcam,Accessories,4,Downtown Store,USA,5,445.0,1,445.0
2024-11-15,7,Multivitemain Tablets,Fitness,5,Green Square Plaza,NewYork,3,897.0,1,897.0
2024-11-12,3,Tab,Electronics,4,Downtown Store,USA,5,4245.0,1,4245.0
2025-03-31,8,Webcam,Accessories,4,Downtown Store,USA,4,356.0,1,356.0
2025-01-04,7,Multivitemain Tablets,Fitness,3,Gachibowli Outlet,Hyderabad,5,1495.0,1,1495.0
